In [0]:
pattern_date_1 = '([0-9]{2}[/]{1}[0-9]{2}[/]{1}[0-9]{4})'
pattern_date_2 = '([0-9]{2}[-]{1}[A-Z]{3}[-]{1}[0-9]{4})'
pattern_date_3 = '([0-9]{2}[-]{1}[A-Z]{3}[-]{1}[0-9]{2})'
pattern_date_4 = '([0-9]{1}[-]{1}[A-Z]{3}[-]{1}[0-9]{2})'

In [0]:
from pyspark.sql import Row
data = [
    ('15/05/2021'),('15-MAY-2021'),('15-MAY-21'),('5-MAY-21'),(None)
]

df = spark.createDataFrame(
   [ Row(date_str = '15/05/2021') ,
    Row(date_str = '15-MAY-2021'),
    Row(date_str = '15-MAY-21'),
    Row(date_str = '5-MAY-21'),
    Row(date_str = None)
   ]
 
)

df.show()

+-----------+
 date_str|
+-----------+
 15/05/2021|
15-MAY-2021|
 15-MAY-21|
 5-MAY-21|
 null|
+-----------+

In [0]:
from pyspark.sql import functions as f
df.filter(
  f.col('date_str').rlike(pattern_date_1)
).show()

+----------+
 date_str|
+----------+
15/05/2021|
+----------+

In [0]:
df.filter(
  f.col('date_str').rlike(pattern_date_2)
).show()

+-----------+
 date_str|
+-----------+
15-MAY-2021|
+-----------+

In [0]:
df.filter(
  f.col('date_str').rlike(pattern_date_3)
).show()

+-----------+
 date_str|
+-----------+
15-MAY-2021|
 15-MAY-21|
+-----------+

In [0]:
df.filter(
  f.col('date_str').rlike(pattern_date_4)
).show()

+-----------+
 date_str|
+-----------+
15-MAY-2021|
 15-MAY-21|
 5-MAY-21|
+-----------+

In [0]:
cond = f.when(
   (
     f.col('date_str').rlike(pattern_date_1) |
     f.col('date_str').rlike(pattern_date_2) |
     f.col('date_str').rlike(pattern_date_3) |
     f.col('date_str').rlike(pattern_date_4)
   ),
  f.concat_ws('',f.to_date(f.col('date_str'),'dd/MM/yyyy'), f.to_date(f.col('date_str'),'dd-MM-yyyy'))
).otherwise(f.col('date_str'))

(
df.withColumn( 'test', cond)
 .withColumn('t1',f.to_date(f.col('date_str'),'dd/MM/yyyy')) 
  .withColumn('t2',f.to_date(f.col('date_str'),'dd-MM-yyyy')) 
).show()

+-----------+----------+----------+----+
 date_str| test| t1| t2|
+-----------+----------+----------+----+
 15/05/2021|2021-05-15|2021-05-15|null|
15-MAY-2021| | null|null|
 15-MAY-21| | null|null|
 5-MAY-21| | null|null|
 null| null| null|null|
+-----------+----------+----------+----+